This tutorial walks you through troubleshooting your own app.py (flask) script.

You should have 3 files downloaded to follow this tutorial:

- app.py 
- home.html 
- result.html

Where you put these files matters. Type this in terminal:

1. ```cd Insight/``` Navigate to your project folder. Mine is called Insight.
2. ``` . venv/bin/activate``` Turn on your virtual environment.
3. ```mkdir templates``` Create a new folder called 'templates'.

Put app.py in your Insight folder.
Put home.html and result.html in your templates/ folder.

*Also important*: We will be using the model.pkl and cat files we created using Random_forests.ipynb. You can ensure these are in the same main directory as app.py by typing in terminal:

```shell
ls -ltr
```
They should appear near the bottom of the terminal window.

## Troubleshooting app.py

Open **app.py** in a text editor. I've copied the important chunks below for troubleshooting, so just observe which chunks I'm working with to make it all go.

#### Import modules (run the cell below)

In [43]:
import pickle
import numpy as np

#### Testing parameters set-by-set

Now what we're doing is running some test cases to ensure that the script and the model are communicating properly. 

**You'll notice that the chunk below is *not* in app.py.** This is because I'm just testing the main portion of app.py, which is pasted immediately below the following cell. 

I'm testing it by giving it a single set of parameters. 

The idea is that I when I manually change these parameters- as a user would do when using the app- the model will spit out *different predictions*, which we can check each time we manually change this single test set.

The alternative parameter options (aka, the categorical values of my Brand, Condition, Category, and Site columns in cleaned_df.csv) are typed as comments below. During troubleshooting, I'll substitute these in manually to ensure the model spits out different predicted prices.

Now, run the cell below.

In [66]:
brand='lululemon' # Reformation
condition='New' # PreOwned
category='bottoms' # tops, bras, dresses, outerwear
site='poshmark' # eBay

Nearly there. Here is what to look for as you adapt the script below for your own work:

1. In ```cat_vector[index_dict['Brand_'+str(brand)]] = 1``` ...you should change **Brand_** and **brand** (or **Condition_** and **condition**, etc.) to correspond to the renamed/one hot encoded categorical columns in your dataframe. Don't know what I mean? See the cell at the bottom where I finally got a clue and ran ```index_dict```, which is created in the third line of the code below. See how the output is a dictionary where the format is: 'Column_option1'? *Don't forget the _ in your string identification*. Forgetting this stole an hour of my life.

2. ```prediction = model.predict(cat_vector.reshape(1, -1))```: Our lord and savior Abhilash helped me with this earlier today. This reshape command is *not* in the [original tutorial](https://xcitech.github.io/tutorials/heroku_tutorial/). It's important!

Run the cell below. 

In [67]:
pkl_file = open('cat', 'rb')
index_dict = pickle.load(pkl_file)
cat_vector = np.zeros(len(index_dict))

try:
    cat_vector[index_dict['Brand_'+str(brand)]] = 1
except:
    pass
try:
    cat_vector[index_dict['Condition_'+str(condition)]] = 1
except:
    pass
try:
    cat_vector[index_dict['Category_'+str(category)]] = 1
except:
    pass
try:
    cat_vector[index_dict['Site_'+str(site)]] = 1
except:
    pass

pkl_file = open('model.pkl', 'rb')
model = pickle.load(pkl_file)
prediction = model.predict(cat_vector.reshape(1, -1))

We can check the prediction each time we rerun the chunk above by printing the prediction:

In [68]:
print(prediction)

[20.]


In [69]:
# I referred to this above...
index_dict

{'Brand_lululemon': 0,
 'Condition_PreOwned': 1,
 'Category_bras': 2,
 'Category_dresses': 3,
 'Category_outerwear': 4,
 'Category_tops': 5,
 'Site_eBay': 6}

**That's it.** You can ensure the prediction changes each time you change the parameters by rerunning the cells in order, 1-2-3.

When you're satisfied that your pickle is talking to the script above, replace the chunk of code in your app.py file with what's above. You'll also have to edit this part of app.py, but it wasn't important for troubleshooting. Just don't forget to edit it:

```python
    if request.method=='POST':
        result=request.form
        brand = result['Brand'] # your Column1
        category = result['Category'] # your Column2 etc.
        condition = result['Condition']
        site = result['Site']
        ```

Deploy your edited app.py in terminal like so:
    
```python
cd Insight
. venv/bin/activate
python app.py
```